In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import os
import nltk
import re
import string
import gensim
import numpy as np

# for tokenization
from nltk.tokenize import word_tokenize
nltk.download("punkt")
nltk.download('wordnet')

# for stopword removal
from nltk.corpus import stopwords
nltk.download('stopwords')

# for lemmatization and POS tagging
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
df = pd.read_csv("/content/drive/MyDrive/train.ft.txt", sep="\t", header=None, nrows=100)
print(df)

                                                    0
0   __label__2 Stuning even for the non-gamer: Thi...
1   __label__2 The best soundtrack ever to anythin...
2   __label__2 Amazing!: This soundtrack is my fav...
3   __label__2 Excellent Soundtrack: I truly like ...
4   __label__2 Remember, Pull Your Jaw Off The Flo...
..                                                ...
95  __label__1 Very Not Worth Your Time: The book ...
96  __label__2 Very fun and educational: Trains, s...
97  __label__1 Ludicrous and silly: I remember get...
98  __label__2 Artistry: I think that the Deodato ...
99  __label__1 Caution!: These tracks are not the ...

[100 rows x 1 columns]


In [5]:
# Split label and text
df[['label', 'text']] = df[0].str.split(' ', n=1, expand=True)

# Remove '__label__' prefix from label
df['label'] = df['label'].str.replace('__label__', '')

# Drop the original column
df.drop(columns=[0], inplace=True)

In [6]:
#converting label values to 0 & 1
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['label'])

In [7]:
valueCounts = df["label"].value_counts().sort_index()
print(valueCounts)

label
0    46
1    54
Name: count, dtype: int64


In [8]:
#PRE-PROCESSING FUNCTION FOR REGULAR EXPRESSION
#import string

#def preprocess_text(text):
    # Convert to lowercase
#    text = text.lower()

    # Remove punctuation marks and special characters
#    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove tokens with numeric values
#    text = ''.join([char for char in text if not char.isdigit()])

    # load NLTK stopword list and add original stopwords
#    stop_words = stopwords.words('english')

    # remove stopwords
#    text = ' '.join([word for word in text.split() if word not in stop_words])

#    return text

#df['text'] = df['text'].apply(preprocess_text)

In [9]:
df

,label,text
0,1,Stuning even for the non-gamer: This sound tra...
1,1,The best soundtrack ever to anything.: I'm rea...
2,1,Amazing!: This soundtrack is my favorite music...
3,1,Excellent Soundtrack: I truly like this soundt...
4,1,"Remember, Pull Your Jaw Off The Floor After He..."
...,...,...
95,0,Very Not Worth Your Time: The book was wriiten...
96,1,"Very fun and educational: Trains, shapes and p..."
97,0,Ludicrous and silly: I remember getting this b...
98,1,Artistry: I think that the Deodato concerts ar...


###SEGREGATING REVIEWS FOR BOOKS

using regular expressions

In [10]:
df['text'] = df['text'].astype(str)
df = df[df['text'].str.contains("book|novel|chapter")]

In [11]:
df

,label,text
6,0,"Buyer beware: This is a self-published book, a..."
7,1,Glorious story: I loved Whisper of the wicked ...
8,1,A FIVE STAR BOOK: I just finished reading Whis...
9,1,Whispers of the Wicked Saints: This was a easy...
10,0,The Worst!: A complete waste of time. Typograp...
11,1,"Great book: This was a great book,I just could..."
12,1,"Great Read: I thought this book was brilliant,..."
13,0,Oh please: I guess you have to be a romance no...
14,0,Awful beyond belief!: I feel I have to write t...
15,0,Don't try to fool us with fake reviews.: It's ...


In [12]:
num_rows = len(df)
print(num_rows)

30


###ABSA

using transformers

In [13]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Load the pre-trained model and tokenizer
model_name = "yangheng/deberta-v3-large-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Create a pipeline for text classification
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Define the aspects
aspects = ["plot", "characters", "writing_style"]

# Define a function to perform ABSA
def absa_analysis(text):
    results = []
    for aspect in aspects:
        result = classifier(text, text_pair=aspect)
        results.append(result)
    return results

# Apply the ABSA function to the text column
df["absa_results"] = df["text"].apply(absa_analysis)

# Explore the results
for index, row in df.iterrows():
    print("Text:", row["text"])
    for aspect, result in zip(aspects, row["absa_results"]):
        print(f"Aspect: {aspect}, Label: {result[0]['label']}, Score: {result[0]['score']}")
    print()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Text: Buyer beware: This is a self-published book, and if you want to know why--read a few paragraphs! Those 5 star reviews must have been written by Ms. Haddon's family and friends--or perhaps, by herself! I can't imagine anyone reading the whole thing--I spent an evening with the book and a friend and we were in hysterics reading bits and pieces of it to one another. It is most definitely bad enough to be entered into some kind of a "worst book" contest. I can't believe Amazon even sells this kind of thing. Maybe I can offer them my 8th grade term paper on "To Kill a Mockingbird"--a book I am quite sure Ms. Haddon never heard of. Anyway, unless you are in a mood to send a book to someone as a joke---stay far, far away from this one!
Aspect: plot, Label: Negative, Score: 0.49974948167800903
Aspect: characters, Label: Negative, Score: 0.5893873572349548
Aspect: writing_style, Label: Negative, Score: 0.6966672539710999

Text: Glorious story: I loved Whisper of the wicked saints. The sto

<ipython-input-13-edad730f6be4>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["absa_results"] = df["text"].apply(absa_analysis)
